In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'price':'{:.2f}','buy_p':'{:.2f}','sell_p':'{:.2f}','diff':'{:.2f}',
               'ppu':'{:.4f}',
               'net':'{:,.2f}','fee':'{:,.2f}','vat':'{:,.2f}',
               'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','profit':'{:,.2f}',
               'div_amt':'{:,.2f}','ttl_amt':'{:,.2f}','pct':'{:,.2f}',
               'qty':'{:,}','number':'{:,}','days':'{:,}'
              }
pd.set_option("display.max_rows", None)

In [2]:
kind = 'DOS'
div_year = 2021
div_quarter = 4
sell_year = 2022

### Stocks that are sold on specified year

In [8]:
sql = '''
SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind, dividend_id
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE S.dividend_id = 0
AND YEAR(s.date) = %s
AND kind = '%s'
ORDER BY T.name, S.date'''
#WHERE status = "Sold" 
sql = sql % (sell_year, kind)
print(sql)


SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind, dividend_id
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE S.dividend_id = 0
AND YEAR(s.date) = 2022
AND kind = 'DOS'
ORDER BY T.name, S.date


In [9]:
sells = pd.read_sql(sql, conpf)
sells.sort_values(['name','sell_date'],ascending=[True,True]).style.format(format_dict)

,name,qty,sell_p,buy_p,diff,sell_amt,buy_amt,profit,sell_date,buy_date,days,pct,id,kind,dividend_id


### Bind sell to dividend record process

In [12]:
sql = '''
UPDATE sells
SET dividend_id = 299
WHERE id = 939'''
rp = conpf.execute(sql)
rp.rowcount

1

### Percent calculation of sells after dividend payment date

In [22]:
sql = """
SELECT name, number, ppu, D.net AS div_amt, profit, profit + D.net AS ttl_amt, B.net AS buy_amt,  
x_date,p_date, B.date AS buy_date, S.date AS sell_date, 
DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS days,
(profit + D.net)/B.net*36500/DATEDIFF(IF(p_date > S.date, p_date, S.date),B.date) AS pct, 
year, quarter
FROM sells S 
JOIN buys B ON S.buy_id = B.id
JOIN dividends D
ON dividend_id = D.id
WHERE B.kind = '%s'
ORDER BY name
"""
#AND YEAR(S.date) = %s
#sql = sql % (kind, sell_year)
sql = sql % (kind)
sells_a_div = pd.read_sql(sql, conpf)
sells_a_div.sort_values(['sell_date'],ascending=[False]).style.format(format_dict)

,name,number,ppu,div_amt,profit,ttl_amt,buy_amt,x_date,p_date,buy_date,sell_date,days,pct,year,quarter
5,LPF,"10,000",0.1719,"1,547.10",377.61,"1,924.71","140,310.09",2022-01-28,2022-02-15,2022-01-14,2022-02-28,45,11.13,2021,4
12,SUPEREIF,"10,000",0.2520,"2,520.40",488.36,"3,008.76","115,254.71",2021-08-19,2021-09-07,2021-08-18,2021-09-02,20,47.64,2021,2
3,GVREIT,"10,000",0.1960,"1,764.00",532.66,"2,296.66","105,232.56",2021-08-23,2021-09-08,2021-05-10,2021-08-24,121,6.58,2021,2
11,SCCC,600,9.0000,"4,860.00","-4,941.88",-81.88,"102,225.92",2021-02-24,2021-04-05,2021-02-23,2021-08-13,171,-0.17,2020,4
10,POPF,"10,000",0.2470,"2,223.00","2,506.08","4,729.08","110,243.64",2021-05-12,2021-06-04,2021-07-30,2021-08-04,5,313.15,2021,1
13,SUPEREIF,"10,000",0.2670,"2,670.00",514.94,"3,184.94","109,241.42",2021-05-21,2021-06-09,2021-05-12,2021-06-08,28,38.01,2021,1
9,POPF,"10,000",0.2470,"2,223.00",514.94,"2,737.94","109,241.42",2021-05-12,2021-06-04,2021-05-06,2021-06-07,32,28.59,2021,1
17,WHART,"9,000",0.1915,"1,551.15",367.77,"1,918.92","119,965.12",2021-05-28,2021-06-14,2021-05-27,2021-05-28,18,32.44,2021,1
2,EGATIF,"10,000",0.2000,"2,000.00","-1,538.22",461.78,"122,270.22",2021-03-01,2021-03-17,2021-02-25,2021-05-13,77,1.79,2020,4
8,PDG,"25,000",0.2200,"4,950.00",-431.92,"4,518.08","97,715.96",2021-05-15,2021-05-20,2021-03-12,2021-05-12,69,24.46,2020,4


In [23]:
total_profit = round(sells_a_div.ttl_amt.sum(),2)
buy_cost = round(sells_a_div.buy_amt.sum(),2)
avg_day = sells_a_div.days.mean()
sell_pct = round(total_profit/buy_cost*36500/avg_day,2)
total_profit,buy_cost,sell_pct

(37593.73, 2289058.84, 11.38)

In [16]:
file_name = 'div-one-spread.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/div-one-spread.csv',
 '\\Users\\User\\iCloudDrive\\div-one-spread.csv',
 '\\Users\\User\\Dropbox\\div-one-spread.csv')

In [17]:
sells_a_div.sort_values(['sell_date','name'],ascending=[False,True]).to_csv(output_file, header=False, index=False)
sells_a_div.sort_values(['sell_date','name'],ascending=[False,True]).to_csv(data_file, header=False, index=False)
sells_a_div.sort_values(['sell_date','name'],ascending=[False,True]).to_csv(box_file, header=False, index=False)

### List DOS stocks that sell before XDATE (do not get dividend)

In [18]:
sql = '''
SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind, S.dividend_id
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE S.dividend_id = 9999
AND YEAR(s.date) = %s
AND kind = '%s'
ORDER BY T.name, S.date'''
#WHERE status = "Sold" 
sql = sql % (sell_year, kind)
print(sql)


SELECT T.name, qty, S.price AS sell_p, B.price AS buy_p, S.price - B.price AS diff,
S.net AS sell_amt, B.net AS buy_amt, S.net - B.net AS profit, 
S.date AS sell_date, B.date AS buy_date, DATEDIFF(S.date,B.date) AS days,
ROUND((S.net-B.net)/B.net*36500/DATEDIFF(S.date,B.date),2) AS pct, S.id, B.kind, S.dividend_id
FROM sells S
JOIN buys B ON S.buy_id = B.id 
JOIN stocks T ON B.stock_id = T.id 
WHERE S.dividend_id = 9999
AND YEAR(s.date) = 2022
AND kind = 'DOS'
ORDER BY T.name, S.date


In [19]:
sells_b_div = pd.read_sql(sql, conpf)
sells_b_div.sort_values(['sell_date'],ascending=[False]).style.format(format_dict)

,name,qty,sell_p,buy_p,diff,sell_amt,buy_amt,profit,sell_date,buy_date,days,pct,id,kind,dividend_id
